<a href="https://colab.research.google.com/github/ShreyaRaundal/1/blob/main/gen1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
latent_dim = 100
batch_size = 64
lr = 0.0002
epochs = 5
image_dir = "gan_images"
os.makedirs(image_dir, exist_ok=True)

# Data loader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
dataloader = DataLoader(
    datasets.MNIST(root='./data', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True
)

# Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(z.size(0), 1, 28, 28)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Training loop
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real_imgs = imgs.to(device)

        valid = torch.ones(imgs.size(0), 1, device=device, dtype=torch.float)
        fake = torch.zeros(imgs.size(0), 1, device=device, dtype=torch.float)

        # ------------------
        # Train Generator
        # ------------------
        optimizer_G.zero_grad()
        z = torch.randn(imgs.size(0), latent_dim, device=device)
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        # Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        if i % 200 == 0:
            print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(dataloader)}] "
                  f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")

    save_image(gen_imgs.data[:25], f"{image_dir}/{epoch:03d}.png", nrow=5, normalize=True)


[Epoch 0/5] [Batch 0/938] [D loss: 0.6504] [G loss: 0.6763]
[Epoch 0/5] [Batch 200/938] [D loss: 0.6293] [G loss: 0.6475]
[Epoch 0/5] [Batch 400/938] [D loss: 0.6551] [G loss: 0.7258]
[Epoch 0/5] [Batch 600/938] [D loss: 0.4301] [G loss: 1.2567]
[Epoch 0/5] [Batch 800/938] [D loss: 0.3410] [G loss: 1.5609]
[Epoch 1/5] [Batch 0/938] [D loss: 1.1355] [G loss: 0.1155]
[Epoch 1/5] [Batch 200/938] [D loss: 0.3005] [G loss: 1.2596]
[Epoch 1/5] [Batch 400/938] [D loss: 0.3025] [G loss: 1.7684]
[Epoch 1/5] [Batch 600/938] [D loss: 0.2579] [G loss: 1.6182]
[Epoch 1/5] [Batch 800/938] [D loss: 0.2326] [G loss: 1.9046]
[Epoch 2/5] [Batch 0/938] [D loss: 0.2533] [G loss: 1.8709]
[Epoch 2/5] [Batch 200/938] [D loss: 0.3014] [G loss: 1.9016]
[Epoch 2/5] [Batch 400/938] [D loss: 0.3281] [G loss: 1.6136]
[Epoch 2/5] [Batch 600/938] [D loss: 0.3340] [G loss: 2.0278]
[Epoch 2/5] [Batch 800/938] [D loss: 0.9347] [G loss: 4.6337]
[Epoch 3/5] [Batch 0/938] [D loss: 0.2778] [G loss: 2.5285]
[Epoch 3/5] [Bat

Epoch 0, Loss: 1.6076
Epoch 100, Loss: 0.0081
Epoch 200, Loss: 0.0035
Epoch 300, Loss: 0.0020
Epoch 400, Loss: 0.0013
Epoch 500, Loss: 0.0009
Epoch 600, Loss: 0.0007
Epoch 700, Loss: 0.0006
Epoch 800, Loss: 0.0004
Epoch 900, Loss: 0.0004
Generated text: hldr  hddelhwdrlrld r
